In [1]:
# Importing Libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Activation
from keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/X-Rays'

Mounted at /content/drive


In [3]:
# Loading the images
train_data_dir = os.path.join(path, 'Train')
test_data_dir = os.path.join(path, 'Test')

In [4]:
def enhance_image(image):
    # Normalize each channel to the range [0, 1]
    normalized = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)

    # Apply contrast stretching to each channel
    stretched = np.zeros_like(normalized)
    for i in range(3):
        channel = normalized[:, :, i]
        min_val = np.min(channel)
        max_val = np.max(channel)
        stretched[:, :, i] = (channel - min_val) / (max_val - min_val)

    # Clip the values to ensure they are within the valid range
    stretched = np.clip(stretched, 0, 1)

    # Convert the image back to uint8 data type
    enhanced_image = (stretched * 255)

    return enhanced_image

In [5]:
# Image Preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=enhance_image)
test_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=enhance_image)

In [6]:
# Training the Model
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='binary',
    shuffle=False
    )

Found 9665 images belonging to 2 classes.
Found 4143 images belonging to 2 classes.


In [7]:
model = Sequential()

#Adding convolutional layers
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Adding a second convolutional layer
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())  # this converts our feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [8]:
# Compiling the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [9]:
# Training the Model
model.fit(
    train_generator,
    steps_per_epoch=32,
    epochs=25,
    validation_data=test_generator,
    validation_steps=32)

Epoch 1/25
32/32 [==============================] - 631s 20s/step - loss: 0.6315 - accuracy: 0.7109 - val_loss: 1.4134 - val_accuracy: 0.0000e+00
Epoch 2/25
32/32 [==============================] - 124s 4s/step - loss: 0.4613 - accuracy: 0.7598 - val_loss: 1.1690 - val_accuracy: 0.1963
Epoch 3/25
32/32 [==============================] - 96s 3s/step - loss: 0.3894 - accuracy: 0.8097 - val_loss: 1.2078 - val_accuracy: 0.0723
Epoch 4/25
32/32 [==============================] - 89s 3s/step - loss: 0.3835 - accuracy: 0.8320 - val_loss: 1.6415 - val_accuracy: 0.1426
Epoch 5/25
32/32 [==============================] - 90s 3s/step - loss: 0.3776 - accuracy: 0.8252 - val_loss: 1.4576 - val_accuracy: 0.1846
Epoch 6/25
32/32 [==============================] - 83s 2s/step - loss: 0.3477 - accuracy: 0.8398 - val_loss: 1.1657 - val_accuracy: 0.3350
Epoch 7/25
32/32 [==============================] - 72s 2s/step - loss: 0.3152 - accuracy: 0.8620 - val_loss: 1.1878 - val_accuracy: 0.3154
Epoch 8/25
32

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 activation (Activation)     (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 72, 72, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 32)        0         
 g2D)                                                   

In [11]:
model.evaluate(test_generator)

130/130 [==============================] - 1171s 9s/step - loss: 0.3130 - accuracy: 0.8530


[0.31297770142555237, 0.853005051612854]

In [12]:
# Saving the Model
model.save('COVID.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
from google.colab import files
files.download('COVID.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>